# Paired integration

In [1]:
# download data
# !mkdir data
# !wget https://cf.10xgenomics.com/samples/cell-arc/2.0.0/10k_PBMC_Multiome_nextgem_Chromium_X/10k_PBMC_Multiome_nextgem_Chromium_X_filtered_feature_bc_matrix.tar.gz
# !cd data; tar -xzf 10k_PBMC_Multiome_nextgem_Chromium_X_filtered_feature_bc_matrix.tar.gz
# !mkdir write

## Environment setup

In [5]:
import scanpy as sc
import anndata as ad
import numpy as np
import pandas as pd
import muon as mu
import anndata2ri
import logging
from torchmetrics.utilities.data import get_num_classes
import scvi
import os
import scipy
import scipy.io
import scib
import rpy2.rinterface_lib.callbacks

import seaborn as sns
import matplotlib.pyplot as plt

from rpy2.robjects import r
from rpy2.robjects import pandas2ri

rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

pandas2ri.activate()
anndata2ri.activate()

%load_ext rpy2.ipython

import warnings

warnings.filterwarnings("ignore")

ImportError: cannot import name 'get_num_classes' from 'torchmetrics.utilities.data' (d:\Anaconda\envs\sc_multiomics\lib\site-packages\torchmetrics\utilities\data.py)

In [3]:
%%R
install.packages('Seurat')
suppressPackageStartupMessages({
    library(Seurat)
})
set.seed(123)

--- Please select a CRAN mirror for use in this session ---
Secure CRAN mirrors 

 1: 0-Cloud [https]
 2: Australia (Canberra) [https]
 3: Australia (Melbourne 1) [https]
 4: Australia (Melbourne 2) [https]
 5: Australia (Perth) [https]
 6: Austria [https]
 7: Belgium (Brussels) [https]
 8: Brazil (PR) [https]
 9: Brazil (RJ) [https]
10: Brazil (SP 1) [https]
11: Brazil (SP 2) [https]
12: Bulgaria [https]
13: Canada (MB) [https]
14: Canada (ON) [https]
15: Chile (Santiago) [https]
16: China (Beijing 2) [https]
17: China (Beijing 3) [https]
18: China (Hefei) [https]
19: China (Hong Kong) [https]
20: China (Guangzhou) [https]
21: China (Jinan) [https]
22: China (Lanzhou) [https]
23: China (Nanjing) [https]
24: China (Shanghai 2) [https]
25: China (Shenzhen) [https]
26: Colombia (Cali) [https]
27: Costa Rica [https]
28: Cyprus [https]
29: Czech Republic [https]
30: Denmark [https]
31: East Asia [https]
32: Ecuador (Cuenca) [https]
33: France (Lyon 1) [https]
34: France (Lyon 2) [https]
35

trying URL 'https://mirrors.nics.utk.edu/cran/bin/macosx/big-sur-arm64/contrib/4.3/Seurat_4.4.0.tgz'
Content type 'application/x-gzip' length 3762575 bytes (3.6 MB)
downloaded 3.6 MB

In addition: Warning message:
In doTryCatch(return(expr), name, parentenv, handler) :
  unable to load shared object '/Library/Frameworks/R.framework/Resources/modules//R_X11.so':
  dlopen(/Library/Frameworks/R.framework/Resources/modules//R_X11.so, 0x0006): Library not loaded: /opt/X11/lib/libSM.6.dylib
  Referenced from: <B3716E5A-BF4D-3CA3-B8EB-89643DB72A04> /Library/Frameworks/R.framework/Versions/4.3-arm64/Resources/modules/R_X11.so
  Reason: tried: '/opt/X11/lib/libSM.6.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/X11/lib/libSM.6.dylib' (no such file), '/opt/X11/lib/libSM.6.dylib' (no such file), '/usr/local/lib/libSM.6.dylib' (no such file), '/usr/lib/libSM.6.dylib' (no such file, not in dyld cache)


## CITE-seq data

We first show how to integrate a CITE-seq dataset using WNN, MOFA+ and totalVI. CITE-seq data contains raw gene expression counts and counts for surface proteins. The surface protein data is represented as antibody-derived tags (adt) here. We refer to the {ref}`surface-protein:motivation` section of the Surface Proteins chapter for more details.

### Prepare data

In [6]:
# adt = sc.read(
#     "/lustre/groups/ml01/workspace/anastasia.litinetskaya/data/neurips-cite/adt_pp.h5ad"
# )
adt = mu.read_10x_mtx(
    'filtered_feature_bc_matrix/',  # the directory with the `.mtx` file
    var_names='gene_symbols',                # use gene symbols for the variable names (variables-axis index)
    cache=True)                              # write a cache file for faster subsequent reading
adt.var_names_make_unique()  # this is unnecessary if using `var_names='gene_ids'` in `sc.read_10x_mtx`
adt

MuData object with n_obs × n_vars = 10970 × 148344
  var:	'gene_ids', 'feature_types'
  2 modalities
    rna:	10970 x 36601
      var:	'gene_ids', 'feature_types'
    atac:	10970 x 111743
      var:	'gene_ids', 'feature_types'

In [ ]:
print("# cells, # genes before filtering:", adata.shape)

sc.pp.filter_genes(adata, min_counts=3)
sc.pp.filter_cells(adata, min_counts=3)

print("# cells, # genes after filtering:", adata.shape)

# cells, # genes before filtering: (10970, 148344)


In [ ]:
rna = sc.read(
    "/lustre/groups/ml01/workspace/anastasia.litinetskaya/data/neurips-cite/rna_hvg.h5ad"
)
rna

We subset the data to Site 1 and the 3 corresponding donors to reduce the run time.

In [ ]:
batches_to_keep = ["s1d1", "s1d2", "s1d3"]
rna = rna[rna.obs["batch"].isin(batches_to_keep)]
adt = adt[adt.obs["donor"].isin(batches_to_keep)]

We only keep the cells that are present in both modality objects. First we need to make sure that `.obs_names` of both objects have similar structure and if not clean up a bit.

In [7]:
adt.obs_names

Index(['AAACAGCCAACAACAA-1', 'AAACAGCCACCGGCTA-1', 'AAACAGCCAGGACACA-1',
       'AAACAGCCATCCTAGA-1', 'AAACATGCAAAGGTAC-1', 'AAACATGCAAATTCGT-1',
       'AAACATGCAACCGCCA-1', 'AAACATGCACTTGTTC-1', 'AAACATGCAGAAATGC-1',
       'AAACATGCAGGACCAA-1',
       ...
       'TTTGTGGCAGGAACTG-1', 'TTTGTGGCAGTTTCTC-1', 'TTTGTGTTCACATTGA-1',
       'TTTGTGTTCGGTACGC-1', 'TTTGTGTTCTAATCCT-1', 'TTTGTGTTCTAGCGTG-1',
       'TTTGTTGGTAAGGTTT-1', 'TTTGTTGGTTAGGATT-1', 'TTTGTTGGTTTGAGCA-1',
       'TTTGTTGGTTTGGGCG-1'],
      dtype='object', length=10970)

In [8]:
rna.obs_names

NameError: name 'rna' is not defined

In [ ]:
adt.obs_names = [
    name.split("-")[0] + "-" + name.split("-")[1] + "-" + batch
    for batch, name in zip(adt.obs["donor"], adt.obs_names)
]

In [ ]:
common_idx = list(set(rna.obs_names).intersection(set(adt.obs_names)))
rna = rna[common_idx].copy()
adt = adt[common_idx].copy()

We need to rename the proteins in the `adt` object so that the gene names and protein names do not intersect.

In [ ]:
adt.var_names = ["PROT_" + name for name in adt.var_names]

Next we create a MuData object where we store data for both modalities.

In [ ]:
mdata = mu.MuData({"rna": rna, "adt": adt})
mdata

We copy `batch` and `cell_type` column from one of the modality adatas to `.obs` of the mdata object to later use for visualizations.

In [ ]:
mdata.obs["batch"] = rna.obs["batch"].copy()
mdata.obs["cell_type"] = rna.obs["cell_type"].copy()

### Weighted Nearest Neighbor (WNN)

WNN is a graph-based method that takes neighbor graphs for each modality and constructs a common graph which is a weighted combination of the modality graphs. This constructed WNN graph can later be used together with gene expression matrix to obtain a supervised PCA (sPCA) representation guided by a WNN graph. The sPCA representation can be viewed as an embedding in a latent space.

First, we use the `anndata2ri` package (https://github.com/theislab/anndata2ri) to move Python AnnData object to SingleCellExperiment and Seurat R objects. We create slimmer versions of AnnData objects that only contain the information that we need to the analysis.

In [ ]:
adata_ = ad.AnnData(adt.X.copy())
adata_.obs_names = adt.obs_names.copy()
adata_.var_names = adt.var_names.copy()
adata_.obs["batch"] = adt.obs["donor"].copy()
adata_.obsm["harmony_pca"] = adt.obsm["X_pcahm"].copy()

In [ ]:
%%R -i adata_
# indicate that data is stored in .X of AnnData object
adt = as.Seurat(adata_, data='X', counts=NULL)
# the assay is called "originalexp" by default, we rename it to "ADT"
adt <- RenameAssays(object = adt, originalexp = "ADT", verbose=FALSE) 
adt

We repeat the same for RNA data.

In [ ]:
adata_ = ad.AnnData(rna.X.copy())
adata_.obs_names = rna.obs_names.copy()
adata_.var_names = rna.var_names.copy()
adata_.obs["cell_type"] = rna.obs["cell_type"].copy()
adata_.obs["batch"] = rna.obs["batch"].copy()

In [ ]:
%%R -i adata_
rna = as.Seurat(adata_, data='X', counts=NULL)
rna

Next we create a Seurat object with both assays.

In [ ]:
%%R
cite <- rna
cite[["ADT"]] <- CreateAssayObject(data = adt@assays$ADT@data)

In [ ]:
%%R
cite <- RenameAssays(object = cite, originalexp = "RNA", verbose=FALSE) 

Since we have several batches in the dataset, we would need to perform batch correction before integrating the modalities. One option would be to batch correct using Seurat's `FindIntegrationAnchors()` and `IntegrateData()` (see https://satijalab.org/seurat/articles/integration_introduction.html) functions separately for each modality. We will use batch corrected embedding from previous analysis done separately for ADT and RNA data, namely Harmony corrected PCA embedding for ADT and scVI batch-corrected latent embedding for RNA.

In [ ]:
%%R
# TODO need to change after we have the preprocessed data, for now RNA is not batch-corrected
DefaultAssay(cite) <- "RNA"
VariableFeatures(cite) <- rownames(cite)
cite <- ScaleData(cite, verbose=FALSE)
cite <- RunPCA(cite, verbose=FALSE)

In [ ]:
%%R
cite@reductions$harmony_pca <- adt@reductions$harmony_pca
cite

Now we follow the WNN vignette to perform the analysis. First, we need to find multimodal neighbors using the specified dimensionality reductions for each of the modalities. This function adds a WNN graph to the Seurat object.

In [ ]:
%%R
cite <- FindMultiModalNeighbors(
    cite, 
    reduction.list = list("pca", "harmony_pca"), 
    dims.list = list(1:50, 1:30), 
    modality.weight.name = "RNA.weight",
    verbose = FALSE
)

Since we are also interested in finding an embedding for our multimodal data, we additionally run the `RunSPCA()` function that uses RNA gene expression data and the WNN graph for supervised PCA. Supervised PCA is a "guided" version of standard PCA run on gene expression data guided by the WNN graph to better preserve relationships between cells learn in WNN graph. We also will need a reference UMAP later for mapping an RNA query onto this multimodal reference as discussed in {ref}`multimodal-integration:advanced-integration`.

In [ ]:
%%R
cite <- RunSPCA(cite, assay = "RNA", graph = "wsnn", npcs = 20)
cite <- RunUMAP(cite, nn.name = "weighted.nn", reduction.name = "wnn.umap", reduction.key = "wnnUMAP_", return.model=TRUE)

In [ ]:
%%R
cite

We save the Seurat object as an `.rds` file for the {ref}`multimodal-integration:advanced-integration` section. 

In [ ]:
%%R
saveRDS(cite, file = "wnn_ref.rds")

We move the sPCA embedding to Python and store them in `.obsm` of the MuData object.

In [ ]:
%%R -o spca
spca = Embeddings(object = cite[["spca"]])

In [ ]:
mdata.obsm["X_spca"] = spca

We also need to extract the calculated WNN graph.

In [ ]:
%%R -o wnn
wnn <- as.data.frame(summary(cite@graphs$wknn))

The table indicates indices with connections between cells in the WNN graph. Since R starts indexing at 1 but Python at 0, we modify the indices to start with 0.

In [ ]:
wnn[:5]

In [ ]:
wnn["i"] = wnn["i"] - 1
wnn["j"] = wnn["j"] - 1
wnn[:5]

We store the graph in `.obsp` of the MuData object.

In [ ]:
mdata.obsp["wnn_connectivities"] = scipy.sparse.coo_matrix(
    (wnn["x"], (wnn["i"], wnn["j"]))
)

Next we use the WWN graph to calculate the UMAP coordinates and save them in `.obsm['X_umap_wnn']`. We could alternatively also just use sPCA coordinates for visualization.

In [ ]:
# we won't actually need the neighbors
# but need to run this anyway as a little trick to make scanpy work with externally-computed neighbors
sc.pp.neighbors(mdata, use_rep="X_spca")
mdata.obsp["connectivities"] = mdata.obsp["wnn_connectivities"].copy()
# delete distances to make sure we are not using anything calculated with sc.pp.neighbors()
del mdata.obsp["distances"]
sc.tl.umap(mdata)

In [ ]:
mdata.obsm["X_umap_wnn"] = mdata.obsm["X_umap"].copy()

Finally we visualize the cell types and batches on a UMAP.

In [ ]:
mu.pl.embedding(
    mdata, color=["cell_type", "batch"], ncols=1, basis="umap_wnn", frameon=False
)

To be able to quantitatively assess the result of the integration and compare to other methods we compute some of the scIB metrics using the sPCA embedding and WNN graph. More specifically, we calculate the following metrics:
- bio conservation: `NMI_cluster/label`, `ARI_cluster/label`, `ASW_label` and `isolated_label_silhouette`;
- batch correction: `ASW_label/batch`, `graph_conn`.

In [ ]:
scib_anndata = sc.AnnData(mdata.obsm["X_spca"]).copy()
scib_anndata.obs = mdata.obs.copy()
scib_anndata.obsp["connectivities"] = mdata.obsp["connectivities"].copy()
scib_anndata.obsm["X_spca"] = mdata.obsm["X_spca"].copy()

In [ ]:
metrics_wnn = scib.metrics.metrics(
    scib_anndata,
    scib_anndata,
    batch_key="batch",
    label_key="cell_type",
    embed="X_spca",
    ari_=True,
    nmi_=True,
    silhouette_=True,
    graph_conn_=True,
    isolated_labels_asw_=True,
)
metrics_wnn

We note that even though batch correction was performed using Harmony for ADT and scVI for RNA, we still include metrics that assess batch correction here too.

### Multi-Omics Factor Analysis (MOFA+)

MOFA+ is a linear factor model that decomposes the input matrices into the product of low-rank matrices. The low-rank representation can be used as an embedding in a low-dimensional space for visualization and other downstream tasks. The latent dimensions are interpretable with respect to the original input features and represent the leading sources of variation in the data.

By default, we are using data from `.X` and the data should be normalized. Since there are some batch effects in the data that MOFA+ can correct for, we also pass the `groups_label` parameter to specify the batch covariate.

If you want to run MOFA+ on a GPU, you need to additionally install a version of cuPY (https://cupy.dev) which is compatible with your CUDA.

In [ ]:
mu.tl.mofa(mdata, groups_label="batch", gpu_mode=True)

We use the `X_mofa` representation to calculate the neighbors and the UMAP coordinates, and store them in `.obsm['X_umap_mofa']`.

In [ ]:
sc.pp.neighbors(mdata, use_rep="X_mofa")
sc.tl.umap(mdata)
mdata.obsm["X_umap_mofa"] = mdata.obsm["X_umap"].copy()

We plot the cell types and batches again on the resulting UMAP.

In [ ]:
mu.pl.embedding(
    mdata, color=["cell_type", "batch"], ncols=1, basis="umap_mofa", frameon=False
)

Finally, we calculate the same scIB metrics as before.

In [ ]:
scib_anndata = sc.AnnData(mdata.obsm["X_mofa"]).copy()
scib_anndata.obs = mdata.obs.copy()
scib_anndata.obsp["connectivities"] = mdata.obsp["connectivities"].copy()
scib_anndata.obsm["X_mofa"] = mdata.obsm["X_mofa"].copy()

In [ ]:
metrics_mofa = scib.metrics.metrics(
    scib_anndata,
    scib_anndata,
    batch_key="batch",
    label_key="cell_type",
    embed="X_mofa",
    ari_=True,
    nmi_=True,
    silhouette_=True,
    graph_conn_=True,
    isolated_labels_asw_=True,
)
metrics_mofa

### Total Variational Inference (totalVI)

TotalVI is variational-inference-based methods for joint analysis of paired gene expression and protein abundance measurements. It takes into account batch effects, protein background noise, which allows the model to learn a join latent representation disentangled form technical factors. TotalVI models transcriptome counts with negative-binomial (NB) distribution and the protein counts as NB mixture of foreground and background signal. Hence, the model takes raw gene expression and raw protein counts as input.

In [ ]:
adata = mdata["rna"].copy()
adata.obsm["protein_expression"] = mdata["adt"].layers["counts"].A.copy()

We need to specify that raw counts for RNA are stored in `counts` layer of our adata and that we want to correct for batch effect with `batch_key="batch"` parameter.

In [ ]:
scvi.model.TOTALVI.setup_anndata(
    adata,
    protein_expression_obsm_key="protein_expression",
    layer="counts",
    batch_key="batch",
)

We initialize the totalVI model.

In [ ]:
vae = scvi.model.TOTALVI(adata)

Next, we train the model with default parameters.

In [ ]:
vae.train()

Next, we obtain the latent representation and store it in `.obsm['X_totalVI']` and then use it to calculate the UMAP coordinates.

In [ ]:
mdata.obsm["X_totalVI"] = vae.get_latent_representation()

In [ ]:
sc.pp.neighbors(mdata, use_rep="X_totalVI")
sc.tl.umap(mdata)

In [ ]:
mdata.obsm["X_umap_totalVI"] = mdata.obsm["X_umap"].copy()

As above, we plot cell types and batches on a UMAP.

In [ ]:
mu.pl.embedding(
    mdata, color=["cell_type", "batch"], ncols=1, basis="umap_totalVI", frameon=False
)

And finally, we calculate scIB metrics.

In [ ]:
scib_anndata = sc.AnnData(mdata.obsm["X_totalVI"]).copy()
scib_anndata.obs = mdata.obs.copy()
scib_anndata.obsp["connectivities"] = mdata.obsp["connectivities"].copy()
scib_anndata.obsm["X_totalVI"] = mdata.obsm["X_totalVI"].copy()

In [ ]:
metrics_totalvi = scib.metrics.metrics(
    scib_anndata,
    scib_anndata,
    batch_key="batch",
    label_key="cell_type",
    embed="X_totalVI",
    ari_=True,
    nmi_=True,
    silhouette_=True,
    graph_conn_=True,
    isolated_labels_asw_=True,
)
metrics_totalvi

### scIB metrics evaluation

To better see the differences in models' performances, we visualize the scIB output for each of the methods. We need to merge the output DataFrames into one and additionally calculate the overall score for each method. We follow the scIB publication and calculate the overall score as `0.4 * batch_correction_metrics + 0.6 * bio_conservation_metrics`.

In [ ]:
metrics = pd.DataFrame([metrics_wnn[0], metrics_mofa[0], metrics_totalvi[0]])
metrics = metrics.set_index(pd.Index(["WNN", "MOFA+", "totalVI"]))
metrics = metrics.dropna(axis=1)
metrics

In [ ]:
metrics["overall"] = (
    0.4 * (metrics["ASW_label/batch"] + metrics["graph_conn"]) / 2
    + 0.6
    * (
        metrics["NMI_cluster/label"]
        + metrics["ARI_cluster/label"]
        + metrics["ASW_label"]
        + metrics["isolated_label_silhouette"]
    )
    / 4
)
metrics

In [ ]:
sns.scatterplot(data=metrics)
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left", borderaxespad=0)

We observe that totalVI obtained the highest overall score, and therefore we will use totalVI embedding later in the notebook to show how one can annotate the cluster in the latent space using both ADT and RNA markers. Depending on the downstream task and the experimental design, selecting the best performing based on a specific metric is advisable.

## Multiome data
To show that integration methods can also work with multiome (i.e. paired RNA-seq and ATAC-seq) data, we demonstrate how multiVI can be used for this task. We note that WNN and MOFA+ can also be run on multiome data with almost exactly the same code as above, so here we only present multiVI where the underlying model differs from totalVI.

### Prepare data

In [ ]:
atac = sc.read(
    "/lustre/groups/ml01/workspace/anastasia.litinetskaya/data/neurips-multiome/atac_hvf.h5ad"
)
atac

In [ ]:
rna = sc.read(
    "/lustre/groups/ml01/workspace/anastasia.litinetskaya/data/neurips-multiome/rna_hvg.h5ad"
)
rna

We again subset the data to one site and 3 batches.

In [ ]:
batches_to_keep = ["s1d1", "s1d2", "s1d3"]
rna = rna[rna.obs["batch"].isin(batches_to_keep)]
atac = atac[atac.obs["batch"].isin(batches_to_keep)]

In [ ]:
mdata_multiome = mu.MuData({"rna": rna, "atac": atac})
mdata_multiome

In [ ]:
mdata_multiome.obs["batch"] = mdata_multiome["rna"].obs["batch"].copy()
mdata_multiome.obs["cell_type"] = mdata_multiome["rna"].obs["cell_type"].copy()

### MultiVI

MultiVI is also based on variational inference and conditional variational autoencoders. The gene expression counts are modeled exactly the same way as in totalVI, i.e. using raw counts and NB distribution. Chromatin accessibility on the other hand is modeled using Bernouli distribution modeling how likely a particular region is to be open. Hence, the input data for ATAC assay has to be binary where 0 means a closed region and 1 means an open region.

In [23]:
adt
n_genes = len(adt["rna"].var_names)
n_regions = len(adt["atac"].var_names)
# n_genes = len(rna.var_names)
# n_regions = len(atac.var_names)

MultiVI requires one AnnData object with concatenated genes and peaks as features. Since we start off with two different objects for each modality but have paired measurements, we can use the following trick to concatenate the AnnData objects along the feature axis.

In [24]:
# adata_paired = ad.concat([rna.copy().T, atac.copy().T]).T
# adata_paired.obs = adata_paired.obs.join(rna.obs[["cell_type", "batch"]])
# adata_paired.obs["modality"] = "paired"
# adata_paired
adata_paired = sc.read_10x_mtx(
    'filtered_feature_bc_matrix/',  # the directory with the `.mtx` file
    var_names='gene_symbols',                # use gene symbols for the variable names (variables-axis index)
    cache=True) 
adata_paired.obs["modality"] = "paired"
adata_paired

AnnData object with n_obs × n_vars = 10970 × 36601
    obs: 'modality'
    var: 'gene_ids', 'feature_types'

In [12]:
adata_mvi = scvi.data.organize_multiome_anndatas(adata_paired)

We also make sure that we pass raw counts as input to the model by specifiying `layer='counts'` in the `setup_anndata` funciton.

In [15]:
scvi.model.MULTIVI.setup_anndata(
    adata_mvi,
    batch_key="modality",
#     categorical_covariate_keys=["batch"],
#     layer="counts",
)

We initialize the MultiVI model.

In [22]:
mvi = scvi.model.MULTIVI(
    adata_mvi,
    n_genes=n_genes,
    n_regions=n_regions,
)

MultiVI Model with INPUTS: n_genes:36601, n_regions:111743, n_proteins:0
n_hidden: 334, n_latent: 18, n_layers_encoder: 2, n_layers_decoder: 2 , dropout_rate: 0.1, latent_distribution: 
normal, deep injection: False, gene_likelihood: zinb, gene_dispersion:gene, Mod.Weights:equal, 
Mod.Penalty:Jeffreys, protein_dispersion:protein
Training status: Not Trained
Model's adata is minified?: False

Next, we train the model with the default parameters.

In [21]:
mvi.train()

TypeError: `devices` selected with `CPUAccelerator` should be an int > 0.

Finally, we visualize the latent embedding on the UMAP.

In [ ]:
mdata_multiome.obsm["X_multiVI"] = mvi.get_latent_representation()

In [ ]:
sc.pp.neighbors(mdata_multiome, use_rep="X_multiVI")
sc.tl.umap(mdata_multiome)

In [ ]:
mdata_multiome.obsm["X_umap_multiVI"] = mdata_multiome.obsm["X_umap"].copy()

In [ ]:
mu.pl.embedding(
    mdata_multiome,
    color=["cell_type", "batch"],
    ncols=1,
    basis="umap_multiVI",
    frameon=False,
)

Session info.

In [ ]:
%%R
sessionInfo()

## References

```{bibliography}
:filter: docname in docnames
```

## Contributors

We gratefully acknowledge the contributions of:

### Authors

* Anastasia Litinetskaya

### Reviewers

* Lukas Heumos